In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 44204  # Number of samples to train on.


In [3]:
data_path="/content/drive/MyDrive/Colab Notebooks/hi.translit.sampled.train.tsv"

In [14]:
data_path1="hi.romanized.rejoined.aligned.cased_nopunct.tsv"

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens+1), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens+1), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens+1), dtype="float32"
)

Number of samples: 44204
Number of unique input tokens: 63
Number of unique output tokens: 28
Max sequence length for inputs: 19
Max sequence length for outputs: 22


In [5]:
target_token_index[" "]=28
input_token_index[" "]=63

In [13]:
input_token_index

{' ': 63,
 'ँ': 0,
 'ं': 1,
 'ः': 2,
 'अ': 3,
 'आ': 4,
 'इ': 5,
 'ई': 6,
 'उ': 7,
 'ऊ': 8,
 'ऋ': 9,
 'ए': 10,
 'ऐ': 11,
 'ऑ': 12,
 'ओ': 13,
 'औ': 14,
 'क': 15,
 'ख': 16,
 'ग': 17,
 'घ': 18,
 'ङ': 19,
 'च': 20,
 'छ': 21,
 'ज': 22,
 'झ': 23,
 'ञ': 24,
 'ट': 25,
 'ठ': 26,
 'ड': 27,
 'ढ': 28,
 'ण': 29,
 'त': 30,
 'थ': 31,
 'द': 32,
 'ध': 33,
 'न': 34,
 'प': 35,
 'फ': 36,
 'ब': 37,
 'भ': 38,
 'म': 39,
 'य': 40,
 'र': 41,
 'ल': 42,
 'व': 43,
 'श': 44,
 'ष': 45,
 'स': 46,
 'ह': 47,
 '़': 48,
 'ा': 49,
 'ि': 50,
 'ी': 51,
 'ु': 52,
 'ू': 53,
 'ृ': 54,
 'ॅ': 55,
 'े': 56,
 'ै': 57,
 'ॉ': 58,
 'ो': 59,
 'ौ': 60,
 '्': 61,
 'ॐ': 62}

In [60]:
np.shape(decoder_input_data)

(44204, 22, 29)

In [61]:
np.shape(decoder_target_data)

(44204, 22, 29)

In [6]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

array([[[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0.

In [21]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens+1))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens+1))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens+1, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [22]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=20,
    validation_split=0.2,
)
# Save model
model.save("s2s1")


Epoch 1/20
553/553 [==============================] - 127s 222ms/step - loss: 1.1227 - accuracy: 0.7006 - val_loss: 0.9914 - val_accuracy: 0.7384
Epoch 2/20
553/553 [==============================] - 125s 226ms/step - loss: 0.6876 - accuracy: 0.7921 - val_loss: 0.9465 - val_accuracy: 0.7594
Epoch 3/20
553/553 [==============================] - 119s 215ms/step - loss: 0.5607 - accuracy: 0.8290 - val_loss: 0.8981 - val_accuracy: 0.7727
Epoch 4/20
553/553 [==============================] - 116s 210ms/step - loss: 0.4660 - accuracy: 0.8556 - val_loss: 0.7805 - val_accuracy: 0.7946
Epoch 5/20
553/553 [==============================] - 111s 202ms/step - loss: 0.3823 - accuracy: 0.8796 - val_loss: 0.6756 - val_accuracy: 0.8147
Epoch 6/20
553/553 [==============================] - 112s 203ms/step - loss: 0.3176 - accuracy: 0.9004 - val_loss: 0.6338 - val_accuracy: 0.8236
Epoch 7/20
553/553 [==============================] - 113s 204ms/step - loss: 0.2681 - accuracy: 0.9150 - val_loss: 0.5583 -

INFO:tensorflow:Assets written to: s2s1/assets


INFO:tensorflow:Assets written to: s2s1/assets


In [26]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s1")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens+1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens+1))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [27]:
for seq_index in range(30):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: अं
Decoded sentence: an

-
Input sentence: अंकगणित
Decoded sentence: angakanit

-
Input sentence: अंकल
Decoded sentence: ankal

-
Input sentence: अंकुर
Decoded sentence: ankur

-
Input sentence: अंकुरण
Decoded sentence: ankuran

-
Input sentence: अंकुरित
Decoded sentence: ankurit

-
Input sentence: अंकुश
Decoded sentence: ankush

-
Input sentence: अंकुश
Decoded sentence: ankush

-
Input sentence: अंग
Decoded sentence: angh

-
Input sentence: अंग
Decoded sentence: angh

-
Input sentence: अंगद
Decoded sentence: angadh

-
Input sentence: अंगद
Decoded sentence: angadh

-
Input sentence: अंगने
Decoded sentence: angane

-
Input sentence: अंगभंग
Decoded sentence: angbhang

-
Input sentence: अंगरक्षक
Decoded sentence: angrakshak

-
Input sentence: अंगरक्षक
Decoded sentence: angrakshak

-
Input sentence: अंगारा
Decoded sentence: angara

-
Input sentence: अंगारे
Decoded sentence: angare

-
Input sentence: अंगारे
Decoded sentence: angare

-
Input sentence: अंगी
Decoded sentence: